# Libraries and Packages

In [12]:
import gensim.corpora as corpora
import pandas as pd

from bertopic import BERTopic
from gensim.models.coherencemodel import CoherenceModel

# Data

In [13]:
file_path = "/Users/catherinepollack/Documents/dartmouth/research/aim3_facebook_covid19_obesity/data"
dat_obesity = "/220331_feature_matrix_no_bert_instagram.csv"

dat_obesity = pd.read_csv(str(file_path + dat_obesity))

/Users/catherinepollack/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (0,1,2,3,99,100,136,140,144,149,150) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [14]:
dat_obesity["date"] = pd.to_datetime(dat_obesity["Post Created Date"], errors = 'coerce')
dat_obesity_restricted = dat_obesity[(dat_obesity['date'] > '2020-01-06') & (dat_obesity['date'] < '2020-10-16')]

In [15]:
dat_obesity_restricted = dat_obesity_restricted.reset_index(drop = True)

In [18]:
dat_obesity_restricted.processed_text_bert = dat_obesity_restricted.processed_text_bert.fillna('')
topic_model = BERTopic(verbose = True, min_topic_size = 100)
topics = topic_model.fit_transform(dat_obesity_restricted.processed_text_bert)

Batches: 100%|██████████| 567/567 [40:28<00:00,  4.28s/it]
2022-03-31 08:22:07,513 - BERTopic - Transformed documents to Embeddings
2022-03-31 08:22:16,888 - BERTopic - Reduced dimensionality with UMAP
2022-03-31 08:22:19,863 - BERTopic - Clustered UMAP embeddings with HDBSCAN


In [19]:
dat_obesity_restricted["topics"] = topics[0]

In [42]:
dat_obesity_restricted.to_csv(file_path + "/220331_obesity_with_topics_instagram.csv")
topic_model.save("220331_bert_topic_model_instagram")

In [24]:
topic_model.get_topic_info()[0:12]

,Topic,Count,Name
0,-1,6856,-1_the_and_to_of
1,0,2718,0_my_to_and_you
2,1,1069,1_the_covid19_of_and
3,2,769,2_cats_pet_to_he
4,3,588,3_keto_carbaddictiondoc_to_carnivore
5,4,415,4_girlz_biggirlzmove_ted_move
6,5,391,5_you_to_calories_the
7,6,363,6_346_10_543_90
8,7,341,7_sugar_of_the_and
9,8,337,8_the_obesity_to_in


In [ ]:
for i in range(0,12):
    print(i)
    docs = topic_model.get_representative_docs()[i]
    for d in docs:
        print(d)
        print("-----")
    print("**********")

In [28]:
topic_model.visualize_hierarchy(topics = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10])

# Topic Coherence

In [30]:
documents = pd.DataFrame({"Document": dat_obesity_restricted.processed_text_bert,
                          "ID":range(len(dat_obesity_restricted.processed_text_bert)),
                          "Topic":  topics[0]})

documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
cleaned_docs = topic_model._preprocess_text(documents_per_topic.Document.values)

In [31]:
vectorizer = topic_model.vectorizer_model
analyzer = vectorizer.build_analyzer()

In [33]:
words = vectorizer.get_feature_names()
tokens = [analyzer(doc) for doc in cleaned_docs]
dictionary = corpora.Dictionary(tokens)
corpus = [dictionary.doc2bow(token) for token in tokens]
topic_words = [[words for words, _ in topic_model.get_topic(topic)] 
               for topic in range(len(set(topics[0]))-1)]

In [40]:
coherence_model = CoherenceModel(topics=topic_words, 
                                 texts=tokens, 
                                 corpus=corpus,
                                 dictionary=dictionary, 
                                 coherence='c_v')
coherence = coherence_model.get_coherence()

In [41]:
coherence

0.5683780153947191